In [1]:
import pandas as pd
import numpy as np
from collections import defaultdict
from collections import Counter
import nltk
import string
from nltk.stem.porter import *
from sklearn import linear_model
from sklearn.feature_extraction import DictVectorizer
import sys
import unicodedata
import string
import operator
import math
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from scipy import sparse
from scipy.spatial.distance import cosine
from sklearn.metrics import average_precision_score
from sklearn.metrics import precision_recall_curve
import matplotlib.pyplot as plt
import importlib
stdout = sys.stdout
importlib.reload(sys)  
#sys.setdefaultencoding('utf8')
sys.stdout = stdout

In [2]:
import jsonlines
import numpy as np
import _pickle as pickle
import os

class Preprocessor:
    def __init__(self, dataDir = './'):
        self.data = []
        self.dataDir = dataDir 

    def isSampleValid(self, sample):
        return sample['gold_label'] != '-'
    
    def getDataFromJSONL(self, filename):
        numInvalid = 0
        with open(filename) as fp:
            reader = jsonlines.Reader(fp)
            for obj in reader.iter(type=dict, skip_invalid=True):
                if self.isSampleValid(obj):
                    sample = {}
                    sample['gold_label'] = obj['gold_label']
                    sample['sentence1'] = obj['sentence1']
                    sample['sentence2'] = obj['sentence2']
                    self.data.append(sample)
                else:
                    numInvalid+=1
        print (len(self.data))
        print (numInvalid)
    
    

In [3]:
pp = Preprocessor()
pp.getDataFromJSONL('./snli_1.0/snli_1.0_train.jsonl')

549367
785


In [4]:
#nltk.download("stopwords")
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.translate.bleu_score import sentence_bleu
stop = set(stopwords.words('english'))
BLEU_Score_List = []
for j in range(len(pp.data)):
    weights = [1./4., 1./4., 1./4., 1./4.]
    sentence1 = pp.data[j]['sentence1']
    sentence1 = unicodedata.normalize('NFKD', sentence1).encode('ascii','ignore')
    reference = [[i for i in sentence1.lower().split() if i not in stop]]
    sentence2 = pp.data[j]['sentence2']
    sentence2 = unicodedata.normalize('NFKD', sentence2).encode('ascii','ignore')
    candidate = [i for i in sentence2.lower().split() if i not in stop]
    length = min([len(reference[0]), len(candidate)]) 
    if length == 0:
        BLEU_Score_List.append(0)
    else:
        if length < 4:
            weights = ( 1. / length ,) * length
        score = sentence_bleu(reference, candidate, weights)
        BLEU_Score_List.append(score)      

c:\users\amulya\appdata\local\programs\python\python37\lib\site-packages\nltk\translate\bleu_score.py:523: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
c:\users\amulya\appdata\local\programs\python\python37\lib\site-packages\nltk\translate\bleu_score.py:523: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
c:\users\amulya\appdata\local\programs\python\python37\lib\site-packages\nltk\translate\bleu_score.py:523: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lowe

In [5]:
%store BLEU_Score_List > BLEU_Score_List.txt

Writing 'BLEU_Score_List' (list) to file 'BLEU_Score_List.txt'.


In [6]:
BLEU_Score_List = eval(open("BLEU_Score_List.txt").read())  
BLEU_Score_Array = np.array(BLEU_Score_List)

In [7]:
BLEU_Score_Array = BLEU_Score_Array[:, np.newaxis]

In [8]:
Target_List = [pp.data[j]['gold_label'] for j in range(len(pp.data))]

In [9]:
%store Target_List > Target_List.txt

Writing 'Target_List' (list) to file 'Target_List.txt'.


In [10]:
Target_List = eval(open("Target_List.txt").read())  
Target_Array = np.array(Target_List)

In [11]:
print (Target_Array.shape)

(549367,)


In [12]:
print (BLEU_Score_Array.shape)
print (BLEU_Score_Array[:10])

(549367, 1)
[[5.79888111e-155]
 [5.39646693e-155]
 [5.56498649e-155]
 [1.38429296e-231]
 [7.81350843e-232]
 [0.00000000e+000]
 [4.31073290e-232]
 [4.77061016e-232]
 [2.61213321e-155]
 [1.52692335e-078]]


In [3]:
vv = Preprocessor()
vv.getDataFromJSONL('./snli_1.0/snli_1.0_dev.jsonl')

549367
785


In [14]:
#nltk.download("stopwords")
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.translate.bleu_score import sentence_bleu
stop = set(stopwords.words('english'))
BLEU_Score_List_Val = []
for j in range(len(vv.data)):
    weights = [1./4., 1./4., 1./4., 1./4.]
    sentence1 = vv.data[j]['sentence1']
    sentence1 = unicodedata.normalize('NFKD', sentence1).encode('ascii','ignore')
    reference = [[i for i in sentence1.lower().split() if i not in stop]]
    sentence2 = vv.data[j]['sentence2']
    sentence2 = unicodedata.normalize('NFKD', sentence2).encode('ascii','ignore')
    candidate = [i for i in sentence2.lower().split() if i not in stop]
    length = min([len(reference[0]), len(candidate)]) 
    if length == 0:
        BLEU_Score_List_Val.append(0)
    else:
        if length < 4:
            weights = ( 1. / length ,) * length
        score = sentence_bleu(reference, candidate, weights)
        BLEU_Score_List_Val.append(score)      

In [ ]:
BLEU_Score_Array_Test = np.array(BLEU_Score_List_Test)
BLEU_Score_Array_Test = BLEU_Score_Array_Test[:, np.newaxis]

In [13]:
tt = Preprocessor()
tt.getDataFromJSONL('./snli_1.0/snli_1.0_test.jsonl')

9824
176


In [14]:
#nltk.download("stopwords")
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.translate.bleu_score import sentence_bleu
stop = set(stopwords.words('english'))
BLEU_Score_List_Test = []
for j in range(len(tt.data)):
    weights = [1./4., 1./4., 1./4., 1./4.]
    sentence1 = tt.data[j]['sentence1']
    sentence1 = unicodedata.normalize('NFKD', sentence1).encode('ascii','ignore')
    reference = [[i for i in sentence1.lower().split() if i not in stop]]
    sentence2 = tt.data[j]['sentence2']
    sentence2 = unicodedata.normalize('NFKD', sentence2).encode('ascii','ignore')
    candidate = [i for i in sentence2.lower().split() if i not in stop]
    length = min([len(reference[0]), len(candidate)]) 
    if length == 0:
        BLEU_Score_List_Test.append(0)
    else:
        if length < 4:
            weights = ( 1. / length ,) * length
        score = sentence_bleu(reference, candidate, weights)
        BLEU_Score_List_Test.append(score)      

In [15]:
%store BLEU_Score_List_Test > BLEU_Score_List_Test.txt

Writing 'BLEU_Score_List_Test' (list) to file 'BLEU_Score_List_Test.txt'.


In [16]:
BLEU_Score_List_Test = eval(open("BLEU_Score_List_Test.txt").read())  
BLEU_Score_Array_Test = np.array(BLEU_Score_List_Test)

In [17]:
BLEU_Score_Array_Test = BLEU_Score_Array_Test[:, np.newaxis]

In [18]:
Target_List_Test = [tt.data[j]['gold_label'] for j in range(len(tt.data))]

In [19]:
%store Target_List_Test > Target_List_Test.txt

Writing 'Target_List_Test' (list) to file 'Target_List_Test.txt'.


In [20]:
Target_List_Test = eval(open("Target_List_Test.txt").read())
Target_Array_Test = np.array(Target_List_Test)

In [21]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
y_pred_train = gnb.fit(BLEU_Score_Array, Target_Array).predict(BLEU_Score_Array)
y_pred_list_train = y_pred_train.tolist()
%store y_pred_list_train > y_pred_list_train.txt

Writing 'y_pred_list_train' (list) to file 'y_pred_list_train.txt'.


In [22]:
print("Number of mislabeled points out of a total %d points : %d" % (BLEU_Score_Array.shape[0],(Target_Array != y_pred_train).sum()))

Number of mislabeled points out of a total 549367 points : 358144


In [23]:
print (549367 - 331625)

217742


In [24]:
print (217742./549367.)

0.396350709088824


In [25]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
y_pred = gnb.fit(BLEU_Score_Array, Target_Array).predict(BLEU_Score_Array_Test)
y_pred_list = y_pred.tolist()
%store y_pred_list > y_pred_list.txt

Writing 'y_pred_list' (list) to file 'y_pred_list.txt'.


In [26]:
print("Number of mislabeled points out of a total %d points : %d" % (BLEU_Score_Array_Test.shape[0],(Target_Array_Test != y_pred).sum()))

Number of mislabeled points out of a total 9824 points : 6367


In [27]:
print (9824 - 5957)

3867


In [28]:
print (3867./9824.)

0.39362785016286644


In [29]:
from sklearn.naive_bayes import MultinomialNB
mnb = MultinomialNB()
y_pred_mnb = mnb.fit(BLEU_Score_Array, Target_Array).predict(BLEU_Score_Array_Test)
y_pred_list_mnb = y_pred_mnb.tolist()
%store y_pred_list_mnb > y_pred_list_mnb.txt

Writing 'y_pred_list_mnb' (list) to file 'y_pred_list_mnb.txt'.


In [30]:
print("Number of mislabeled points out of a total %d points : %d" % (BLEU_Score_Array_Test.shape[0],(Target_Array_Test != y_pred_mnb).sum()))

Number of mislabeled points out of a total 9824 points : 6456


In [31]:
print (9824 - 6456)

3368


In [32]:
print (3368./9824.)

0.34283387622149836


In [33]:
from sklearn import linear_model
clf = linear_model.SGDClassifier()
y_pred_svm_train = clf.fit(BLEU_Score_Array, Target_Array).predict(BLEU_Score_Array)
y_pred_list_svm_train = y_pred_svm_train.tolist()
%store y_pred_list_svm_train > y_pred_list_svm_train.txt

Writing 'y_pred_list_svm_train' (list) to file 'y_pred_list_svm_train.txt'.


In [34]:
print("Number of mislabeled points out of a total %d points : %d" % (BLEU_Score_Array.shape[0],(Target_Array != y_pred_svm_train).sum()))

Number of mislabeled points out of a total 549367 points : 366603


In [35]:
print (549367 - 361681)

187686


In [36]:
print (187686./549367.)

0.34164046984984536


In [37]:
from sklearn import linear_model
clf = linear_model.SGDClassifier()
y_pred_svm = clf.fit(BLEU_Score_Array, Target_Array).predict(BLEU_Score_Array_Test)
y_pred_list_svm = y_pred_svm.tolist()
%store y_pred_list_svm > y_pred_list_svm.txt

Writing 'y_pred_list_svm' (list) to file 'y_pred_list_svm.txt'.


In [38]:
print("Number of mislabeled points out of a total %d points : %d" % (BLEU_Score_Array_Test.shape[0],(Target_Array_Test != y_pred_svm).sum()))

Number of mislabeled points out of a total 9824 points : 6473


In [39]:
print (9824 - 6587)

3237


In [40]:
print (3237./9824.)

0.32949918566775244


In [41]:
from sklearn import linear_model
clf = linear_model.LogisticRegression()
y_pred_lr_train = clf.fit(BLEU_Score_Array, Target_Array).predict(BLEU_Score_Array)
y_pred_list_lr_train = y_pred_lr_train.tolist()
%store y_pred_list_lr_train > y_pred_list_lr_train.txt

c:\users\amulya\appdata\local\programs\python\python37\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
c:\users\amulya\appdata\local\programs\python\python37\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


Writing 'y_pred_list_lr_train' (list) to file 'y_pred_list_lr_train.txt'.


In [42]:
print("Number of mislabeled points out of a total %d points : %d" % (BLEU_Score_Array.shape[0],(Target_Array != y_pred_lr_train).sum()))

Number of mislabeled points out of a total 549367 points : 356679


In [43]:
print (549367 - 344628)

204739


In [44]:
print (204739./549367.)

0.3726816499716947


In [45]:
from sklearn import linear_model
clf = linear_model.LogisticRegression()
y_pred_lr = clf.fit(BLEU_Score_Array, Target_Array).predict(BLEU_Score_Array_Test)
y_pred_list_lr = y_pred_lr.tolist()
%store y_pred_list_lr > y_pred_list_lr.txt

c:\users\amulya\appdata\local\programs\python\python37\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
c:\users\amulya\appdata\local\programs\python\python37\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


Writing 'y_pred_list_lr' (list) to file 'y_pred_list_lr.txt'.


In [46]:
print("Number of mislabeled points out of a total %d points : %d" % (BLEU_Score_Array_Test.shape[0],(Target_Array_Test != y_pred_lr).sum()))

Number of mislabeled points out of a total 9824 points : 6330


In [47]:
print (9824 - 6198)

3626


In [48]:
print (3626./9824.)

0.36909609120521175


In [49]:
from sklearn import metrics
print(metrics.classification_report(Target_Array_Test, y_pred))

               precision    recall  f1-score   support

contradiction       0.34      0.94      0.50      3237
   entailment       0.47      0.12      0.19      3368
      neutral       0.00      0.00      0.00      3219

     accuracy                           0.35      9824
    macro avg       0.27      0.35      0.23      9824
 weighted avg       0.27      0.35      0.23      9824



c:\users\amulya\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [50]:
from sklearn import metrics
print(metrics.classification_report(Target_Array_Test, y_pred_mnb))

               precision    recall  f1-score   support

contradiction       0.00      0.00      0.00      3237
   entailment       0.34      1.00      0.51      3368
      neutral       0.00      0.00      0.00      3219

     accuracy                           0.34      9824
    macro avg       0.11      0.33      0.17      9824
 weighted avg       0.12      0.34      0.18      9824



In [51]:
from sklearn import metrics
print(metrics.classification_report(Target_Array_Test, y_pred_svm))

               precision    recall  f1-score   support

contradiction       0.00      0.00      0.00      3237
   entailment       0.47      0.12      0.19      3368
      neutral       0.33      0.91      0.48      3219

     accuracy                           0.34      9824
    macro avg       0.26      0.35      0.23      9824
 weighted avg       0.27      0.34      0.22      9824



In [52]:
from sklearn import metrics
print(metrics.classification_report(Target_Array_Test, y_pred_lr))

               precision    recall  f1-score   support

contradiction       0.34      0.93      0.50      3237
   entailment       0.48      0.14      0.22      3368
      neutral       0.00      0.00      0.00      3219

     accuracy                           0.36      9824
    macro avg       0.27      0.36      0.24      9824
 weighted avg       0.28      0.36      0.24      9824



In [53]:
metrics.confusion_matrix(Target_Array_Test, y_pred)


array([[3044,  193,    0],
       [2955,  413,    0],
       [2938,  281,    0]], dtype=int64)

In [54]:
metrics.confusion_matrix(Target_Array_Test, y_pred_lr)

array([[3017,  220,    0],
       [2891,  477,    0],
       [2924,  295,    0]], dtype=int64)

In [55]:
Length_Difference_List = []
for j in range(len(pp.data)):
    sentence1 = pp.data[j]['sentence1']
    sentence1 = unicodedata.normalize('NFKD', sentence1).encode('ascii','ignore')
    premise = [i for i in sentence1.lower().split()]
    sentence2 = pp.data[j]['sentence2']
    sentence2 = unicodedata.normalize('NFKD', sentence2).encode('ascii','ignore')
    hypothesis = [i for i in sentence2.lower().split()]
    length_diff = len(premise)-len(hypothesis)
    Length_Difference_List.append(length_diff)    

In [56]:
%store Length_Difference_List > Length_Difference_List.txt

Writing 'Length_Difference_List' (list) to file 'Length_Difference_List.txt'.


In [57]:
Length_Difference_List = eval(open("Length_Difference_List.txt").read())  
Length_Difference_Array = np.array(Length_Difference_List)

In [58]:
Length_Difference_Array = Length_Difference_Array[:, np.newaxis]

In [59]:
Two_Features = np.concatenate((BLEU_Score_Array, Length_Difference_Array), axis=1)

In [60]:
print (Two_Features.shape)

(549367, 2)


In [61]:
Length_Difference_List_Test = []
for j in range(len(tt.data)):
    sentence1 = tt.data[j]['sentence1']
    sentence1 = unicodedata.normalize('NFKD', sentence1).encode('ascii','ignore')
    premise = [i for i in sentence1.lower().split()]
    sentence2 = tt.data[j]['sentence2']
    sentence2 = unicodedata.normalize('NFKD', sentence2).encode('ascii','ignore')
    hypothesis = [i for i in sentence2.lower().split()]
    length_diff = len(premise)-len(hypothesis)
    Length_Difference_List_Test.append(length_diff)    

In [62]:
%store Length_Difference_List_Test > Length_Difference_List_Test.txt

Writing 'Length_Difference_List_Test' (list) to file 'Length_Difference_List_Test.txt'.


In [63]:
Length_Difference_List_Test = eval(open("Length_Difference_List_Test.txt").read())
Length_Difference_Array_Test = np.array(Length_Difference_List_Test)

In [64]:
Length_Difference_Array_Test = Length_Difference_Array_Test[:, np.newaxis]

In [65]:
Two_Features_Test = np.concatenate((BLEU_Score_Array_Test, Length_Difference_Array_Test), axis=1)

In [66]:
print (Two_Features_Test.shape)

(9824, 2)


In [67]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
y_pred_2_train = gnb.fit(Two_Features, Target_Array).predict(Two_Features)
y_pred_2_list_train = y_pred_2_train.tolist()
%store y_pred_2_list_train > y_pred_2_list_train.txt

Writing 'y_pred_2_list_train' (list) to file 'y_pred_2_list_train.txt'.


In [68]:
print("Number of mislabeled points out of a total %d points : %d" % (Two_Features.shape[0],(Target_Array != y_pred_2_train).sum()))

Number of mislabeled points out of a total 549367 points : 343682


In [69]:
print (549367 - 329733)

219634


In [70]:
print (219634./549367.)

0.3997946727779426


In [71]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
y_pred_2 = gnb.fit(Two_Features, Target_Array).predict(Two_Features_Test)
y_pred_2_list = y_pred_2.tolist()
%store y_pred_2_list > y_pred_2_list.txt

Writing 'y_pred_2_list' (list) to file 'y_pred_2_list.txt'.


In [72]:
print("Number of mislabeled points out of a total %d points : %d" % (Two_Features_Test.shape[0],(Target_Array_Test != y_pred_2).sum()))

Number of mislabeled points out of a total 9824 points : 6094


In [73]:
print (9824 - 5897)

3927


In [74]:
print (3927./9824.)

0.39973534201954397


In [75]:
from sklearn import metrics
print(metrics.classification_report(Target_Array_Test, y_pred_2))

               precision    recall  f1-score   support

contradiction       0.34      0.85      0.49      3237
   entailment       0.55      0.12      0.20      3368
      neutral       0.52      0.18      0.27      3219

     accuracy                           0.38      9824
    macro avg       0.47      0.38      0.32      9824
 weighted avg       0.47      0.38      0.32      9824



In [76]:
metrics.confusion_matrix(Target_Array_Test, y_pred_2)


array([[2749,  166,  322],
       [2749,  400,  219],
       [2477,  161,  581]], dtype=int64)

In [77]:
from sklearn import linear_model
clf = linear_model.SGDClassifier()
y_pred_2_svm_train = clf.fit(Two_Features, Target_Array).predict(Two_Features)
y_pred_list_svm_2_train = y_pred_2_svm_train.tolist()
%store y_pred_list_svm_2_train > y_pred_list_svm_2_train.txt

Writing 'y_pred_list_svm_2_train' (list) to file 'y_pred_list_svm_2_train.txt'.


In [78]:
print("Number of mislabeled points out of a total %d points : %d" % (Two_Features.shape[0],(Target_Array != y_pred_2_svm_train).sum()))

Number of mislabeled points out of a total 549367 points : 353298


In [79]:
print (549367 - 336877)

212490


In [80]:
print (212490./549367.)

0.3867906153809748


In [81]:
from sklearn import linear_model
clf = linear_model.SGDClassifier()
y_pred_2_svm = clf.fit(Two_Features, Target_Array).predict(Two_Features_Test)
y_pred_list_svm_2 = y_pred_2_svm.tolist()
%store y_pred_list_svm_2 > y_pred_list_svm_2.txt

Writing 'y_pred_list_svm_2' (list) to file 'y_pred_list_svm_2.txt'.


In [82]:
print("Number of mislabeled points out of a total %d points : %d" % (Two_Features_Test.shape[0],(Target_Array_Test != y_pred_2_svm).sum()))

Number of mislabeled points out of a total 9824 points : 6255


In [83]:
print (9824 - 6005)

3819


In [84]:
print (3819./9824.)

0.3887418566775244


In [85]:
from sklearn import metrics
print(metrics.classification_report(Target_Array_Test, y_pred_2_svm))

               precision    recall  f1-score   support

contradiction       0.28      0.09      0.13      3237
   entailment       0.37      0.87      0.52      3368
      neutral       0.37      0.11      0.17      3219

     accuracy                           0.36      9824
    macro avg       0.34      0.36      0.27      9824
 weighted avg       0.34      0.36      0.28      9824



In [86]:
metrics.confusion_matrix(Target_Array_Test, y_pred_2_svm)


array([[ 277, 2608,  352],
       [ 171, 2942,  255],
       [ 533, 2336,  350]], dtype=int64)

In [87]:
from sklearn import linear_model
clf = linear_model.LogisticRegression()
y_pred_2_lr_train = clf.fit(Two_Features, Target_Array).predict(Two_Features)
y_pred_list_lr_2_train = y_pred_2_lr_train.tolist()
%store y_pred_list_lr_2_train > y_pred_list_lr_2_train.txt

c:\users\amulya\appdata\local\programs\python\python37\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
c:\users\amulya\appdata\local\programs\python\python37\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


Writing 'y_pred_list_lr_2_train' (list) to file 'y_pred_list_lr_2_train.txt'.


In [88]:
print("Number of mislabeled points out of a total %d points : %d" % (Two_Features.shape[0],(Target_Array != y_pred_2_lr_train).sum()))

Number of mislabeled points out of a total 549367 points : 343526


In [89]:
print (549367 - 337283)

212084


In [90]:
print (212084./549367.)

0.38605158300371156


In [91]:
from sklearn import linear_model
clf = linear_model.LogisticRegression()
y_pred_2_lr = clf.fit(Two_Features, Target_Array).predict(Two_Features_Test)
y_pred_list_lr_2 = y_pred_2_lr.tolist()
%store y_pred_list_lr_2 > y_pred_list_lr_2.txt

c:\users\amulya\appdata\local\programs\python\python37\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
c:\users\amulya\appdata\local\programs\python\python37\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


Writing 'y_pred_list_lr_2' (list) to file 'y_pred_list_lr_2.txt'.


In [92]:
print("Number of mislabeled points out of a total %d points : %d" % (Two_Features_Test.shape[0],(Target_Array_Test != y_pred_2_lr).sum()))

Number of mislabeled points out of a total 9824 points : 6117


In [93]:
print (9824 - 6029)

3795


In [94]:
print (3795./9824.)

0.3862988599348534


In [95]:
from sklearn import metrics
print(metrics.classification_report(Target_Array_Test, y_pred_2_lr))

               precision    recall  f1-score   support

contradiction       0.33      0.17      0.23      3237
   entailment       0.39      0.48      0.43      3368
      neutral       0.38      0.47      0.42      3219

     accuracy                           0.38      9824
    macro avg       0.37      0.38      0.36      9824
 weighted avg       0.37      0.38      0.36      9824



In [96]:
#nltk.download("stopwords")
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.translate.bleu_score import sentence_bleu
stop = set(stopwords.words('english'))
ps = PorterStemmer()
Overlap_Percent = []
for j in range(len(pp.data)):
    sentence1 = pp.data[j]['sentence1']
    #sentence1 = unicodedata.normalize('NFKD', sentence1).encode('ascii','ignore')
    sentence1 = unicodedata.normalize('NFKD', sentence1)
    premise = [ps.stem(i) for i in sentence1.lower().split() if i not in stop]
    sentence2 = pp.data[j]['sentence2']
    sentence2 = unicodedata.normalize('NFKD', sentence2)
    hypothesis = [ps.stem(i) for i in sentence2.lower().split() if i not in stop]
    lengtharray = np.array([len(premise), len(hypothesis)])
    length = np.min(lengtharray)
    indx = np.argmin(lengtharray)
    if length == 0:
        Overlap_Percent.append(0)
    else:
        count = 0
        if indx == 0:
            for j in range(length):
                if premise[j] in hypothesis:
                    count = count+1
        else:
            for j in range(length):
                if hypothesis[j] in premise:
                    count = count+1
        Overlap_Percent.append(count/float(length))                        

In [97]:
%store Overlap_Percent > Overlap_Percent.txt

Writing 'Overlap_Percent' (list) to file 'Overlap_Percent.txt'.


In [98]:
Overlap_Percent = eval(open("Overlap_Percent.txt").read())
Overlap_Percent_Array = np.array(Overlap_Percent)

In [99]:
Overlap_Percent_Array = Overlap_Percent_Array[:, np.newaxis]

In [100]:
#nltk.download("stopwords")
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.translate.bleu_score import sentence_bleu
stop = set(stopwords.words('english'))
ps = PorterStemmer()
Overlap_Percent_Test = []
for j in range(len(tt.data)):
    sentence1 = tt.data[j]['sentence1']
    sentence1 = unicodedata.normalize('NFKD', sentence1)
    premise = [ps.stem(i) for i in sentence1.lower().split() if i not in stop]
    sentence2 = tt.data[j]['sentence2']
    sentence2 = unicodedata.normalize('NFKD', sentence2)
    hypothesis = [ps.stem(i) for i in sentence2.lower().split() if i not in stop]
    lengtharray = np.array([len(premise), len(hypothesis)])
    length = np.min(lengtharray)
    indx = np.argmin(lengtharray)
    if length == 0:
        Overlap_Percent_Test.append(0)
    else:
        count = 0
        if indx == 0:
            for j in range(length):
                if premise[j] in hypothesis:
                    count = count+1
        else:
            for j in range(length):
                if hypothesis[j] in premise:
                    count = count+1
        Overlap_Percent_Test.append(count/float(length))  

In [101]:
%store Overlap_Percent_Test > Overlap_Percent_Test.txt

Writing 'Overlap_Percent_Test' (list) to file 'Overlap_Percent_Test.txt'.


In [102]:
Overlap_Percent_Test = eval(open("Overlap_Percent_Test.txt").read())
Overlap_Percent_Array_Test = np.array(Overlap_Percent_Test)

In [103]:
Overlap_Percent_Array_Test = Overlap_Percent_Array_Test[:, np.newaxis]

In [104]:
Three_Features = np.concatenate((Two_Features, Overlap_Percent_Array), axis=1)

In [105]:
print (Three_Features.shape)

(549367, 3)


In [106]:
Three_Features_Test = np.concatenate((Two_Features_Test, Overlap_Percent_Array_Test), axis=1)

In [107]:
print (Three_Features_Test.shape)

(9824, 3)


In [108]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
y_pred_3_train = gnb.fit(Three_Features, Target_Array).predict(Three_Features)
y_pred_3_list_train = y_pred_3_train.tolist()
%store y_pred_3_list_train > y_pred_3_list_train.txt

Writing 'y_pred_3_list_train' (list) to file 'y_pred_3_list_train.txt'.


In [109]:
print("Number of mislabeled points out of a total %d points : %d" % (Three_Features.shape[0],(Target_Array != y_pred_3_train).sum()))

Number of mislabeled points out of a total 549367 points : 297560


In [110]:
print (549367-293646)

255721


In [111]:
print (255721./549367.)

0.46548300134518455


In [112]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
y_pred_3 = gnb.fit(Three_Features, Target_Array).predict(Three_Features_Test)
y_pred_3_list = y_pred_3.tolist()
%store y_pred_3_list > y_pred_3_list.txt

Writing 'y_pred_3_list' (list) to file 'y_pred_3_list.txt'.


In [113]:
print("Number of mislabeled points out of a total %d points : %d" % (Three_Features_Test.shape[0],(Target_Array_Test != y_pred_3).sum()))

Number of mislabeled points out of a total 9824 points : 5288


In [114]:
print (9824 - 5174)

4650


In [115]:
print (4650./9824.)

0.47333061889250816


In [116]:
from sklearn import metrics
print(metrics.classification_report(Target_Array_Test, y_pred_3))

               precision    recall  f1-score   support

contradiction       0.42      0.67      0.51      3237
   entailment       0.55      0.52      0.54      3368
      neutral       0.43      0.18      0.26      3219

     accuracy                           0.46      9824
    macro avg       0.47      0.46      0.44      9824
 weighted avg       0.47      0.46      0.44      9824



In [117]:
metrics.confusion_matrix(Target_Array_Test, y_pred_3)


array([[2183,  654,  400],
       [1222, 1766,  380],
       [1838,  794,  587]], dtype=int64)

In [118]:
from sklearn import linear_model
clf = linear_model.SGDClassifier()
y_pred_3_svm_train = clf.fit(Three_Features, Target_Array).predict(Three_Features)
y_pred_list_svm_3_train = y_pred_3_svm_train.tolist()
%store y_pred_list_svm_3_train > y_pred_list_svm_3_train.txt

Writing 'y_pred_list_svm_3_train' (list) to file 'y_pred_list_svm_3_train.txt'.


In [119]:
print("Number of mislabeled points out of a total %d points : %d" % (Three_Features.shape[0],(Target_Array != y_pred_3_svm_train).sum()))

Number of mislabeled points out of a total 549367 points : 304885


In [120]:
print (549367 - 314507)

234860


In [121]:
print (234860./549367.)

0.42751020720210714


In [122]:
from sklearn import linear_model
clf = linear_model.SGDClassifier()
y_pred_3_svm = clf.fit(Three_Features, Target_Array).predict(Three_Features_Test)
y_pred_list_svm_3 = y_pred_3_svm.tolist()
%store y_pred_list_svm_3 > y_pred_list_svm_3.txt

Writing 'y_pred_list_svm_3' (list) to file 'y_pred_list_svm_3.txt'.


In [123]:
print("Number of mislabeled points out of a total %d points : %d" % (Three_Features_Test.shape[0],(Target_Array_Test != y_pred_3_svm).sum()))

Number of mislabeled points out of a total 9824 points : 5700


In [124]:
print (9824 - 5304)

4520


In [125]:
print (4520./9824.)

0.4600977198697068


In [126]:
from sklearn import metrics
print(metrics.classification_report(Target_Array_Test, y_pred_3_svm))

               precision    recall  f1-score   support

contradiction       0.43      0.34      0.38      3237
   entailment       0.42      0.80      0.55      3368
      neutral       0.38      0.11      0.17      3219

     accuracy                           0.42      9824
    macro avg       0.41      0.41      0.37      9824
 weighted avg       0.41      0.42      0.37      9824



In [127]:
metrics.confusion_matrix(Target_Array_Test, y_pred_3_svm)


array([[1088, 1774,  375],
       [ 489, 2695,  184],
       [ 931, 1947,  341]], dtype=int64)

In [128]:
from sklearn import linear_model
clf = linear_model.LogisticRegression()
y_pred_3_lr_train = clf.fit(Three_Features, Target_Array).predict(Three_Features)
y_pred_list_lr_3_train = y_pred_3_lr_train.tolist()
%store y_pred_list_lr_3_train > y_pred_list_lr_3_train.txt

c:\users\amulya\appdata\local\programs\python\python37\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
c:\users\amulya\appdata\local\programs\python\python37\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


Writing 'y_pred_list_lr_3_train' (list) to file 'y_pred_list_lr_3_train.txt'.


In [129]:
print("Number of mislabeled points out of a total %d points : %d" % (Three_Features.shape[0],(Target_Array != y_pred_3_lr_train).sum()))

Number of mislabeled points out of a total 549367 points : 295417


In [130]:
print (549367 - 297395)

251972


In [131]:
print (251972./549367.)

0.4586587836546425


In [132]:
from sklearn import linear_model
clf = linear_model.LogisticRegression()
y_pred_3_lr = clf.fit(Three_Features, Target_Array).predict(Three_Features_Test)
y_pred_list_lr_3 = y_pred_3_lr.tolist()
%store y_pred_list_lr_3 > y_pred_list_lr_3.txt

c:\users\amulya\appdata\local\programs\python\python37\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
c:\users\amulya\appdata\local\programs\python\python37\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


Writing 'y_pred_list_lr_3' (list) to file 'y_pred_list_lr_3.txt'.


In [133]:
print("Number of mislabeled points out of a total %d points : %d" % (Three_Features_Test.shape[0],(Target_Array_Test != y_pred_3_lr).sum()))

Number of mislabeled points out of a total 9824 points : 5258


In [134]:
print (9824 - 5253)

4571


In [135]:
print (4571./9824.)

0.46528908794788276


In [136]:
from sklearn import metrics
print(metrics.classification_report(Target_Array_Test, y_pred_3_lr))

               precision    recall  f1-score   support

contradiction       0.44      0.53      0.48      3237
   entailment       0.50      0.63      0.56      3368
      neutral       0.43      0.23      0.30      3219

     accuracy                           0.46      9824
    macro avg       0.46      0.46      0.45      9824
 weighted avg       0.46      0.46      0.45      9824



In [137]:
from sklearn import metrics
print(metrics.classification_report(Target_Array_Test, y_pred_3_lr))

               precision    recall  f1-score   support

contradiction       0.44      0.53      0.48      3237
   entailment       0.50      0.63      0.56      3368
      neutral       0.43      0.23      0.30      3219

     accuracy                           0.46      9824
    macro avg       0.46      0.46      0.45      9824
 weighted avg       0.46      0.46      0.45      9824



In [138]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
#nltk.download('vader_lexicon')

In [139]:
sid = SentimentIntensityAnalyzer()

In [140]:
sentence1 = pp.data[2]['sentence1']

In [141]:
ss = sid.polarity_scores(sentence1)

In [142]:
print(ss)

{'neg': 0.307, 'neu': 0.693, 'pos': 0.0, 'compound': -0.4767}


In [143]:
print(sentence1)

A person on a horse jumps over a broken down airplane.


In [144]:
sentence2 = pp.data[2]['sentence2']

In [145]:
ss = sid.polarity_scores(sentence2)

In [146]:
print(ss)

{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}


In [147]:
print (sentence2)

A person is outdoors, on a horse.


In [148]:
Premise_Sentiment = []
Hypothesis_Sentiment = []
Match = []
for j in range(len(pp.data)):
    sentence1 = pp.data[j]['sentence1']
    premise_ss = sid.polarity_scores(sentence1)
    Premise_Sentiment.append(premise_ss['compound'])
    sentence2 = pp.data[j]['sentence2']
    hypothesis_ss = sid.polarity_scores(sentence2)
    Hypothesis_Sentiment.append(hypothesis_ss['compound'])
    if premise_ss['compound'] >= 0 and hypothesis_ss['compound'] >= 0:
        Match.append(1)
    elif premise_ss['compound'] >=0 and hypothesis_ss['compound'] < 0:
        Match.append(-1)
    elif premise_ss['compound'] < 0 and hypothesis_ss['compound'] >= 0:
        Match.append(-1)
    else:
        Match.append(1)
%store Premise_Sentiment > Premise_Sentiment.txt
%store Hypothesis_Sentiment > Hypothesis_Sentiment.txt
%store Match > Match.txt

Writing 'Premise_Sentiment' (list) to file 'Premise_Sentiment.txt'.
Writing 'Hypothesis_Sentiment' (list) to file 'Hypothesis_Sentiment.txt'.
Writing 'Match' (list) to file 'Match.txt'.


In [149]:
Premise_Sentiment = eval(open("Premise_Sentiment.txt").read())
Premise_Sentiment_Array = np.array(Premise_Sentiment)
Hypothesis_Sentiment = eval(open("Hypothesis_Sentiment.txt").read())
Hypothesis_Sentiment_Array = np.array(Hypothesis_Sentiment)
Match = eval(open("Match.txt").read())
Match_Array = np.array(Match)

In [150]:
Premise_Sentiment_Array = Premise_Sentiment_Array[:, np.newaxis]

In [151]:
Hypothesis_Sentiment_Array = Hypothesis_Sentiment_Array[:, np.newaxis]

In [152]:
Match_Array = Match_Array[:, np.newaxis]

In [153]:
Six_Features = np.concatenate((Three_Features, Premise_Sentiment_Array, Hypothesis_Sentiment_Array, Match_Array), axis=1)

In [154]:
print(Six_Features.shape)

(549367, 6)


In [155]:
Premise_Sentiment_Test = []
Hypothesis_Sentiment_Test = []
Match_Test = []
for j in range(len(tt.data)):
    sentence1 = tt.data[j]['sentence1']
    premise_ss = sid.polarity_scores(sentence1)
    Premise_Sentiment_Test.append(premise_ss['compound'])
    sentence2 = tt.data[j]['sentence2']
    hypothesis_ss = sid.polarity_scores(sentence2)
    Hypothesis_Sentiment_Test.append(hypothesis_ss['compound'])
    if premise_ss['compound'] >= 0 and hypothesis_ss['compound'] >= 0:
        Match_Test.append(1)
    elif premise_ss['compound'] >=0 and hypothesis_ss['compound'] < 0:
        Match_Test.append(-1)
    elif premise_ss['compound'] < 0 and hypothesis_ss['compound'] >= 0:
        Match_Test.append(-1)
    else:
        Match_Test.append(1)
%store Premise_Sentiment_Test > Premise_Sentiment_Test.txt
%store Hypothesis_Sentiment_Test > Hypothesis_Sentiment_Test.txt
%store Match_Test > Match_Test.txt

Writing 'Premise_Sentiment_Test' (list) to file 'Premise_Sentiment_Test.txt'.
Writing 'Hypothesis_Sentiment_Test' (list) to file 'Hypothesis_Sentiment_Test.txt'.
Writing 'Match_Test' (list) to file 'Match_Test.txt'.


In [156]:
Premise_Sentiment_Test = eval(open("Premise_Sentiment_Test.txt").read())
Premise_Sentiment_Array_Test = np.array(Premise_Sentiment_Test)
Hypothesis_Sentiment_Test = eval(open("Hypothesis_Sentiment_Test.txt").read())
Hypothesis_Sentiment_Array_Test = np.array(Hypothesis_Sentiment_Test)
Match_Test = eval(open("Match_Test.txt").read())
Match_Array_Test = np.array(Match_Test)

In [157]:
Premise_Sentiment_Array_Test = Premise_Sentiment_Array_Test[:, np.newaxis]
Hypothesis_Sentiment_Array_Test = Hypothesis_Sentiment_Array_Test[:, np.newaxis]
Match_Array_Test = Match_Array_Test[:, np.newaxis]

In [158]:
Six_Features_Test = np.concatenate((Three_Features_Test, Premise_Sentiment_Array_Test, Hypothesis_Sentiment_Array_Test, Match_Array_Test), axis=1)

In [159]:
print (Six_Features_Test.shape)

(9824, 6)


In [160]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
y_pred_6 = gnb.fit(Six_Features, Target_Array).predict(Six_Features_Test)
y_pred_6_list = y_pred_6.tolist()
%store y_pred_6_list > y_pred_6_list.txt

Writing 'y_pred_6_list' (list) to file 'y_pred_6_list.txt'.


In [161]:
print("Number of mislabeled points out of a total %d points : %d" % (Six_Features_Test.shape[0],(Target_Array_Test != y_pred_6).sum()))

Number of mislabeled points out of a total 9824 points : 5175


In [162]:
from sklearn import linear_model
clf = linear_model.SGDClassifier()
y_pred_6_svm = clf.fit(Six_Features, Target_Array).predict(Six_Features_Test)
y_pred_list_svm_6 = y_pred_6_svm.tolist()
%store y_pred_list_svm_6 > y_pred_list_svm_6.txt

Writing 'y_pred_list_svm_6' (list) to file 'y_pred_list_svm_6.txt'.


In [163]:
print("Number of mislabeled points out of a total %d points : %d" % (Six_Features_Test.shape[0],(Target_Array_Test != y_pred_6_svm).sum()))

Number of mislabeled points out of a total 9824 points : 5168


In [164]:
from sklearn import linear_model
clf = linear_model.LogisticRegression()
y_pred_6_lr = clf.fit(Six_Features, Target_Array).predict(Six_Features_Test)
y_pred_list_lr_6 = y_pred_6_lr.tolist()
%store y_pred_list_lr_6 > y_pred_list_lr_6.txt

c:\users\amulya\appdata\local\programs\python\python37\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
c:\users\amulya\appdata\local\programs\python\python37\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


Writing 'y_pred_list_lr_6' (list) to file 'y_pred_list_lr_6.txt'.


In [165]:
print("Number of mislabeled points out of a total %d points : %d" % (Six_Features_Test.shape[0],(Target_Array_Test != y_pred_6_lr).sum()))

Number of mislabeled points out of a total 9824 points : 5060


In [166]:
print (9824 - 5092)

4732


In [167]:
print (4732./9824.)

0.4816775244299674


In [168]:
Five_Features = np.concatenate((Three_Features, Hypothesis_Sentiment_Array, Match_Array), axis=1)

In [169]:
Five_Features_Test = np.concatenate((Three_Features_Test, Hypothesis_Sentiment_Array_Test, Match_Array_Test), axis=1)

In [170]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
y_pred_5 = gnb.fit(Five_Features, Target_Array).predict(Five_Features_Test)
y_pred_5_list = y_pred_5.tolist()
%store y_pred_5_list > y_pred_5_list.txt

Writing 'y_pred_5_list' (list) to file 'y_pred_5_list.txt'.


In [171]:
print("Number of mislabeled points out of a total %d points : %d" % (Five_Features_Test.shape[0],(Target_Array_Test != y_pred_5).sum()))

Number of mislabeled points out of a total 9824 points : 5184


In [172]:
from sklearn import linear_model
clf = linear_model.SGDClassifier()
y_pred_5_svm = clf.fit(Five_Features, Target_Array).predict(Five_Features_Test)
y_pred_list_svm_5 = y_pred_5_svm.tolist()
%store y_pred_list_svm_5 > y_pred_list_svm_5.txt

Writing 'y_pred_list_svm_5' (list) to file 'y_pred_list_svm_5.txt'.


In [173]:
print("Number of mislabeled points out of a total %d points : %d" % (Five_Features_Test.shape[0],(Target_Array_Test != y_pred_5_svm).sum()))

Number of mislabeled points out of a total 9824 points : 5344


In [174]:
from sklearn import linear_model
clf = linear_model.LogisticRegression()
y_pred_5_lr = clf.fit(Five_Features, Target_Array).predict(Five_Features_Test)
y_pred_list_lr_5 = y_pred_5_lr.tolist()
%store y_pred_list_lr_5 > y_pred_list_lr_5.txt

c:\users\amulya\appdata\local\programs\python\python37\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
c:\users\amulya\appdata\local\programs\python\python37\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


Writing 'y_pred_list_lr_5' (list) to file 'y_pred_list_lr_5.txt'.


In [175]:
print("Number of mislabeled points out of a total %d points : %d" % (Five_Features_Test.shape[0],(Target_Array_Test != y_pred_5_lr).sum()))

Number of mislabeled points out of a total 9824 points : 5078


In [176]:
Four_Features_H = np.concatenate((Three_Features, Hypothesis_Sentiment_Array), axis=1)

In [177]:
Four_Features_Test_H = np.concatenate((Three_Features_Test, Hypothesis_Sentiment_Array_Test), axis=1)

In [178]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
y_pred_4_h = gnb.fit(Four_Features_H, Target_Array).predict(Four_Features_Test_H)
y_pred_4_list_h = y_pred_4_h.tolist()
%store y_pred_4_list_h > y_pred_4_list_h.txt

Writing 'y_pred_4_list_h' (list) to file 'y_pred_4_list_h.txt'.


In [179]:
print("Number of mislabeled points out of a total %d points : %d" % (Four_Features_Test_H.shape[0],(Target_Array_Test != y_pred_4_h).sum()))

Number of mislabeled points out of a total 9824 points : 5085


In [180]:
print (9824 - 5025)

4799


In [181]:
print (4799./9824.)

0.4884975570032573


In [182]:
from sklearn import linear_model
clf = linear_model.SGDClassifier()
y_pred_4_svm_h = clf.fit(Four_Features_H, Target_Array).predict(Four_Features_Test_H)
y_pred_list_svm_4_h = y_pred_4_svm_h.tolist()
%store y_pred_list_svm_4_h > y_pred_list_svm_4_h.txt

Writing 'y_pred_list_svm_4_h' (list) to file 'y_pred_list_svm_4_h.txt'.


In [183]:
print("Number of mislabeled points out of a total %d points : %d" % (Four_Features_Test_H.shape[0],(Target_Array_Test != y_pred_4_svm_h).sum()))

Number of mislabeled points out of a total 9824 points : 5401


In [184]:
from sklearn import linear_model
clf = linear_model.LogisticRegression()
y_pred_4_lr_h = clf.fit(Four_Features_H, Target_Array).predict(Four_Features_Test_H)
y_pred_list_lr_4_h = y_pred_4_lr_h.tolist()
%store y_pred_list_lr_4_h > y_pred_list_lr_4_h.txt

c:\users\amulya\appdata\local\programs\python\python37\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
c:\users\amulya\appdata\local\programs\python\python37\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


Writing 'y_pred_list_lr_4_h' (list) to file 'y_pred_list_lr_4_h.txt'.


In [185]:
print("Number of mislabeled points out of a total %d points : %d" % (Four_Features_Test_H.shape[0],(Target_Array_Test != y_pred_4_lr_h).sum()))

Number of mislabeled points out of a total 9824 points : 5052


In [186]:
Four_Features_M = np.concatenate((Three_Features, Match_Array), axis=1)

In [187]:
Four_Features_Test_M = np.concatenate((Three_Features_Test, Match_Array_Test), axis=1)

In [188]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
y_pred_4_m = gnb.fit(Four_Features_H, Target_Array).predict(Four_Features_Test_H)
y_pred_4_list_m = y_pred_4_m.tolist()
%store y_pred_4_list_m > y_pred_4_list_m.txt

Writing 'y_pred_4_list_m' (list) to file 'y_pred_4_list_m.txt'.


In [189]:
print("Number of mislabeled points out of a total %d points : %d" % (Four_Features_Test_M.shape[0],(Target_Array_Test != y_pred_4_m).sum()))

Number of mislabeled points out of a total 9824 points : 5085


In [190]:
from sklearn import linear_model
clf = linear_model.SGDClassifier()
y_pred_4_svm_m = clf.fit(Four_Features_M, Target_Array).predict(Four_Features_Test_M)
y_pred_list_svm_4_m = y_pred_4_svm_m.tolist()
%store y_pred_list_svm_4_m > y_pred_list_svm_4_m.txt

Writing 'y_pred_list_svm_4_m' (list) to file 'y_pred_list_svm_4_m.txt'.


In [191]:
print("Number of mislabeled points out of a total %d points : %d" % (Four_Features_Test_M.shape[0],(Target_Array_Test != y_pred_4_svm_m).sum()))

Number of mislabeled points out of a total 9824 points : 5411


In [192]:
from sklearn import linear_model
clf = linear_model.LogisticRegression()
y_pred_4_lr_m = clf.fit(Four_Features_M, Target_Array).predict(Four_Features_Test_M)
y_pred_list_lr_4_m = y_pred_4_lr_m.tolist()
%store y_pred_list_lr_4_m > y_pred_list_lr_4_m.txt

c:\users\amulya\appdata\local\programs\python\python37\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
c:\users\amulya\appdata\local\programs\python\python37\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


Writing 'y_pred_list_lr_4_m' (list) to file 'y_pred_list_lr_4_m.txt'.


In [193]:
print("Number of mislabeled points out of a total %d points : %d" % (Four_Features_Test_H.shape[0],(Target_Array_Test != y_pred_4_lr_m).sum()))

Number of mislabeled points out of a total 9824 points : 5236


In [194]:
#nltk.download("stopwords")
from nltk import word_tokenize
from nltk.corpus import stopwords
stop = set(stopwords.words('english'))
ps = PorterStemmer()
wordCount = defaultdict(int)
for j in range(len(pp.data)):
    sentence2 = pp.data[j]['sentence2']
    sentence2 = unicodedata.normalize('NFKD', sentence2)
    hypothesis = [ps.stem(i) for i in sentence2.lower().split() if i not in stop]
    for w in hypothesis:
        wordCount[w]+=1
    



In [195]:
#%store wordCount > wordCount.txt

In [196]:
# Limit the vocabulary
#wordCount = eval(open("wordCount.txt").read())
uni_sorted = sorted(wordCount.items(), key=operator.itemgetter(1)) [::-1]
vocab = [item[0] for item in uni_sorted[:10000]]
print (vocab[:100])

['man', 'peopl', 'woman', 'two', 'play', 'girl', 'boy', 'dog', 'men', 'sit', 'wear', 'person', 'walk', 'stand', 'group', 'women', 'young', 'child', 'outside.', 'ride', 'look', 'hold', 'run', 'three', 'children', 'watch', 'kid', 'outsid', 'eat', 'take', 'get', 'work', 'black', 'near', 'wait', 'jump', 'player', 'white', 'ladi', 'front', 'coupl', 'red', 'one', 'go', 'blue', 'guy', 'dress', 'perform', 'littl', 'shirt', 'sleep', 'street.', 'make', 'bike', 'old', 'street', 'talk', 'tri', 'water.', 'worker', 'swim', 'outdoors.', 'beach.', 'next', 'someon', 'car', 'crowd', 'water', 'pictur', 'insid', 'use', 'around', 'park.', 'danc', 'enjoy', 'friend', 'human', 'drink', 'babi', 'larg', 'read', 'four', 'ball', 'small', 'game', 'build', 'park', 'carri', 'cat', 'soccer', 'game.', 'green', 'race', 'together.', 'drive', 'pose', 'climb', 'anim', 'famili', 'prepar']


In [197]:
print (len(vocab))

10000


In [198]:
#nltk.download("stopwords")
from nltk import word_tokenize
from nltk.corpus import stopwords
stop = set(stopwords.words('english'))
ps = PorterStemmer()
Char_Vector_List = []
for j in range(len(pp.data)):
    sentence2 = pp.data[j]['sentence2']
    sentence2 = unicodedata.normalize('NFKD', sentence2)
    hypothesis = [ps.stem(i) for i in sentence2.lower().split() if i not in stop]
    L = []
    for w in hypothesis:
        if w in vocab:
            index = vocab.index(w)
            L.append(index)
    Char_Vector_List.append(L)                    

In [199]:
%store Char_Vector_List > Char_Vector_List.txt

Writing 'Char_Vector_List' (list) to file 'Char_Vector_List.txt'.


In [200]:
print (Char_Vector_List[0])

[11, 148, 108, 306]


In [201]:
print (len(Char_Vector_List))

549367


In [202]:
from scipy.sparse import csr_matrix
Char_Vector_List = eval(open("Char_Vector_List.txt").read())
indptr = [0]
length = 0
indices = []
for d in Char_Vector_List:
    length = length + len(d)
    indptr.append(length)
    for j in range(len(d)):
        indices.append(d[j])
data = np.ones(length)        
X = csr_matrix((data, indices, indptr), dtype=int)

In [203]:
print (X.shape)
print(type(X))

(549367, 10000)
<class 'scipy.sparse.csr.csr_matrix'>


In [204]:
Three_Features_Sparse = csr_matrix(Three_Features)

In [205]:
from scipy.sparse import hstack
Many_Features = hstack([Three_Features_Sparse,X])

In [206]:
print (Many_Features.shape)

(549367, 10003)


In [207]:
#nltk.download("stopwords")
from nltk import word_tokenize
from nltk.corpus import stopwords
stop = set(stopwords.words('english'))
ps = PorterStemmer()
Char_Vector_Array_Test = np.zeros((len(tt.data),len(vocab)))
for j in range(len(tt.data)):
    sentence2 = tt.data[j]['sentence2']
    sentence2 = unicodedata.normalize('NFKD', sentence2)
    hypothesis = [ps.stem(i) for i in sentence2.lower().split() if i not in stop]
    L = []
    for w in hypothesis:
        if w in vocab:
            index = vocab.index(w)
            Char_Vector_Array_Test[j,index] = 1       

In [208]:
print (Char_Vector_Array_Test.shape)

(9824, 10000)


In [209]:
Many_Features_Test = np.concatenate((Three_Features_Test, Char_Vector_Array_Test), axis=1)

In [210]:
print (Many_Features_Test.shape)

(9824, 10003)


In [211]:
from sklearn import linear_model
clf = linear_model.LogisticRegression()
y_pred_many_lr_train = clf.fit(Many_Features, Target_Array).predict(Many_Features)
y_pred_list_many_lr_train = y_pred_many_lr_train.tolist()
%store y_pred_list_many_lr_train > y_pred_list_many_lr_train.txt

c:\users\amulya\appdata\local\programs\python\python37\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
c:\users\amulya\appdata\local\programs\python\python37\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


Writing 'y_pred_list_many_lr_train' (list) to file 'y_pred_list_many_lr_train.txt'.


In [212]:
print("Number of mislabeled points out of a total %d points : %d" % (Many_Features.shape[0],(Target_Array != y_pred_many_lr_train).sum()))

Number of mislabeled points out of a total 549367 points : 184674


In [213]:
print (549367 - 184547)

364820


In [214]:
print (364820./549367.)

0.6640733789980104


In [215]:
from sklearn import linear_model
clf = linear_model.LogisticRegression()
y_pred_many_lr = clf.fit(Many_Features, Target_Array).predict(Many_Features_Test)
y_pred_list_many_lr = y_pred_many_lr.tolist()
%store y_pred_list_many_lr > y_pred_list_many_lr.txt

Writing 'y_pred_list_many_lr' (list) to file 'y_pred_list_many_lr.txt'.


In [216]:
print("Number of mislabeled points out of a total %d points : %d" % (Many_Features_Test.shape[0],(Target_Array_Test != y_pred_many_lr).sum()))

Number of mislabeled points out of a total 9824 points : 3290


In [217]:
print (9824 - 3301)

6523


In [218]:
print (6523./9824)

0.6639861563517915


In [219]:
from sklearn import metrics
print(metrics.classification_report(Target_Array_Test, y_pred_many_lr))

               precision    recall  f1-score   support

contradiction       0.65      0.67      0.66      3237
   entailment       0.67      0.74      0.70      3368
      neutral       0.68      0.59      0.63      3219

     accuracy                           0.67      9824
    macro avg       0.67      0.66      0.66      9824
 weighted avg       0.67      0.67      0.66      9824



In [220]:
metrics.confusion_matrix(Target_Array_Test, y_pred_many_lr)

array([[2160,  587,  490],
       [ 468, 2486,  414],
       [ 675,  656, 1888]], dtype=int64)

In [221]:
from sklearn import linear_model
clf = linear_model.SGDClassifier()
y_pred_many_svm_train = clf.fit(Many_Features, Target_Array).predict(Many_Features)
y_pred_list_many_svm_train = y_pred_many_svm_train.tolist()
%store y_pred_list_many_svm_train > y_pred_list_many_svm_train.txt

Writing 'y_pred_list_many_svm_train' (list) to file 'y_pred_list_many_svm_train.txt'.


In [222]:
print("Number of mislabeled points out of a total %d points : %d" % (Many_Features.shape[0],(Target_Array != y_pred_many_svm_train).sum()))

Number of mislabeled points out of a total 549367 points : 209330


In [223]:
print (549367 - 203710)

345657


In [224]:
print (345657./549367.)

0.6291914148465416


In [225]:
from sklearn import linear_model
clf = linear_model.SGDClassifier()
y_pred_many_svm = clf.fit(Many_Features, Target_Array).predict(Many_Features_Test)
y_pred_list_many_svm = y_pred_many_svm.tolist()
%store y_pred_list_many_svm > y_pred_list_many_svm.txt

Writing 'y_pred_list_many_svm' (list) to file 'y_pred_list_many_svm.txt'.


In [226]:
print("Number of mislabeled points out of a total %d points : %d" % (Many_Features_Test.shape[0],(Target_Array_Test != y_pred_many_svm).sum()))

Number of mislabeled points out of a total 9824 points : 3560


In [227]:
print (9824 - 3838)

5986


In [228]:
print (5986./9824.)

0.6093241042345277


### New code Begins

In [229]:
from sklearn import linear_model
# Regularized regression
clf = linear_model.RidgeClassifier(1.0, fit_intercept=False) # MSE + 1.0 l2
y_pred_many_rr = clf.fit(Many_Features, Target_Array).predict(Many_Features_Test)
y_pred_list_many_rr = y_pred_many_rr.tolist()
%store y_pred_list_many_rr > y_pred_list_many_rr.txt


Writing 'y_pred_list_many_rr' (list) to file 'y_pred_list_many_rr.txt'.


In [230]:
print("Number of mislabeled points out of a total %d points : %d" % (Many_Features_Test.shape[0],(Target_Array_Test != y_pred_many_rr).sum()))

Number of mislabeled points out of a total 9824 points : 3354


In [231]:
print(9824 - 3311)

6513


In [232]:
print(6513/9824)

0.6629682410423453


### Cosine Similarity + Bag of words

In [ ]:
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.translate.bleu_score import sentence_bleu
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity


stop = set(stopwords.words('english'))
ps = PorterStemmer()
vectorizer = CountVectorizer()
cosine_sim_vector = []

#all_words = defaultdict(int)
#prem_hyp_pair = []
X =[]
for j in range(len(pp.data)):
    sentence1 = pp.data[j]['sentence1']
    sentence1 = unicodedata.normalize('NFKD', sentence1)
    premise = [ps.stem(i) for i in sentence1.lower().split() if i not in stop]
    #for word in premise:
    #    all_words[word] += 1
    sentence2 = pp.data[j]['sentence2']
    sentence2 = unicodedata.normalize('NFKD', sentence2)
    hypothesis = [ps.stem(i) for i in sentence2.lower().split() if i not in stop]
    #for word in hypothesis:
    #    all_words[word] += 1
    allsentences = []
    allsentences.append(' '.join(premise))
    allsentences.append(' '.join(hypothesis))
    #prem_hyp_pair.append((premise,hypothesis))
    
    sparse_matrix = vectorizer.fit_transform(allsentences)
    doc_term_matrix = sparse_matrix.todense()
    
    cosine_sim_vector.append(cosine_similarity(doc_term_matrix, doc_term_matrix)[0][1])

In [ ]:
Cosine_Sim_Array = np.array(cosine_sim_vector)
Cosine_Sim_Array = Cosine_Sim_Array[:, np.newaxis]

Cos_Three_Features = np.concatenate((Two_Features, Cosine_Sim_Array), axis=1)
print(Cos_Three_Features.shape)
print(Cosine_Sim_Array.shape, Two_Features.shape)

In [ ]:
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.translate.bleu_score import sentence_bleu
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity


stop = set(stopwords.words('english'))
ps = PorterStemmer()
vectorizer = CountVectorizer()
cosine_sim_vector_test = []

#all_words = defaultdict(int)
#prem_hyp_pair = []
X =[]
for j in range(len(tt.data)):
    sentence1 = tt.data[j]['sentence1']
    sentence1 = unicodedata.normalize('NFKD', sentence1)
    premise = [ps.stem(i) for i in sentence1.lower().split() if i not in stop]
    #for word in premise:
    #    all_words[word] += 1
    sentence2 = tt.data[j]['sentence2']
    sentence2 = unicodedata.normalize('NFKD', sentence2)
    hypothesis = [ps.stem(i) for i in sentence2.lower().split() if i not in stop]
    #for word in hypothesis:
    #    all_words[word] += 1
    allsentences = []
    allsentences.append(' '.join(premise))
    allsentences.append(' '.join(hypothesis))
    #prem_hyp_pair.append((premise,hypothesis))
    
    sparse_matrix = vectorizer.fit_transform(allsentences)
    doc_term_matrix = sparse_matrix.todense()
    
    cosine_sim_vector_test.append(cosine_similarity(doc_term_matrix, doc_term_matrix)[0][1])

In [ ]:
Cosine_Sim_Array_Test = np.array(cosine_sim_vector_test)
Cosine_Sim_Array_Test = Cosine_Sim_Array_Test[:, np.newaxis]

Cos_Three_Features_Test = np.concatenate((Two_Features_Test, Cosine_Sim_Array_Test), axis=1)
print(Cos_Three_Features_Test.shape)

In [ ]:
from sklearn import linear_model
clf = linear_model.LogisticRegression()
y_pred_cos_three_lr = clf.fit(Cos_Three_Features, Target_Array).predict(Cos_Three_Features_Test)
y_pred_list_cos_three_lr = y_pred_cos_three_lr.tolist()
#%store y_pred_list_cos_three_lr > y_pred_list_cos_three_lr.txt

In [ ]:
total = Cos_Three_Features_Test.shape[0]
mislabelled = (Target_Array_Test != y_pred_cos_three_lr).sum()
print("Number of mislabeled points out of a total %d points : %d" % (Cos_Three_Features_Test.shape[0],(Target_Array_Test != y_pred_cos_three_lr).sum()))

In [ ]:
acc = (total-mislabelled) / total
print("Accuracy {}".format(acc))

In [ ]:
### Using Many features along with cosine similarity

from scipy.sparse import hstack
Cos_Many_Features = hstack([Many_Features,Cosine_Sim_Array])
print(Many_Features.shape, Cosine_Sim_Array.shape)

print(Cos_Many_Features.shape)

In [ ]:
Cos_Many_Features_Test = np.concatenate((Many_Features_Test, Cosine_Sim_Array_Test), axis=1)
print(Cos_Many_Features_Test.shape)

In [ ]:
from sklearn import linear_model
clf = linear_model.LogisticRegression()
y_pred_cos_many_lr = clf.fit(Cos_Many_Features, Target_Array).predict(Cos_Many_Features_Test)
y_pred_list_cos_many_lr = y_pred_cos_many_lr.tolist()

In [ ]:
total = Cos_Many_Features_Test.shape[0]
mislabelled = (Target_Array_Test != y_pred_cos_many_lr).sum()
print("Number of mislabeled points out of a total %d points : %d" % (Cos_Many_Features_Test.shape[0],(Target_Array_Test != y_pred_cos_many_lr).sum()))

In [ ]:
acc = (total-mislabelled) / total
print("Accuracy {}".format(acc))

### Cosine Similarity + TF-IDF

In [ ]:
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.translate.bleu_score import sentence_bleu
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity


stop = set(stopwords.words('english'))
ps = PorterStemmer()
vectorizer = TfidfVectorizer()
cosine_sim_vector = []

#all_words = defaultdict(int)
#prem_hyp_pair = []
for j in range(len(pp.data)):
    sentence1 = pp.data[j]['sentence1']
    sentence1 = unicodedata.normalize('NFKD', sentence1)
    premise = [ps.stem(i) for i in sentence1.lower().split() if i not in stop]
    #for word in premise:
    #    all_words[word] += 1
    sentence2 = pp.data[j]['sentence2']
    sentence2 = unicodedata.normalize('NFKD', sentence2)
    hypothesis = [ps.stem(i) for i in sentence2.lower().split() if i not in stop]
    #for word in hypothesis:
    #    all_words[word] += 1
    allsentences = []
    allsentences.append(' '.join(premise))
    allsentences.append(' '.join(hypothesis))
    #prem_hyp_pair.append((premise,hypothesis))
    
    sparse_matrix = vectorizer.fit_transform(allsentences)
    doc_term_matrix = sparse_matrix.todense()
    
    cosine_sim_vector.append(cosine_similarity(doc_term_matrix, doc_term_matrix)[0][1])

In [ ]:
Cosine_Sim_Array = np.array(cosine_sim_vector)
Cosine_Sim_Array = Cosine_Sim_Array[:, np.newaxis]

Cos_Three_Features = np.concatenate((Two_Features, Cosine_Sim_Array), axis=1)
print(Cos_Three_Features.shape)
print(Cosine_Sim_Array.shape, Two_Features.shape)

In [ ]:
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.translate.bleu_score import sentence_bleu
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity


stop = set(stopwords.words('english'))
ps = PorterStemmer()
vectorizer = TfidfVectorizer()
cosine_sim_vector_test = []

#all_words = defaultdict(int)
#prem_hyp_pair = []
X =[]
for j in range(len(tt.data)):
    sentence1 = tt.data[j]['sentence1']
    sentence1 = unicodedata.normalize('NFKD', sentence1)
    premise = [ps.stem(i) for i in sentence1.lower().split() if i not in stop]
    #for word in premise:
    #    all_words[word] += 1
    sentence2 = tt.data[j]['sentence2']
    sentence2 = unicodedata.normalize('NFKD', sentence2)
    hypothesis = [ps.stem(i) for i in sentence2.lower().split() if i not in stop]
    #for word in hypothesis:
    #    all_words[word] += 1
    allsentences = []
    allsentences.append(' '.join(premise))
    allsentences.append(' '.join(hypothesis))
    #prem_hyp_pair.append((premise,hypothesis))
    
    sparse_matrix = vectorizer.fit_transform(allsentences)
    doc_term_matrix = sparse_matrix.todense()
    
    cosine_sim_vector_test.append(cosine_similarity(doc_term_matrix, doc_term_matrix)[0][1])

In [ ]:
Cosine_Sim_Array_Test = np.array(cosine_sim_vector_test)
Cosine_Sim_Array_Test = Cosine_Sim_Array_Test[:, np.newaxis]

Cos_Three_Features_Test = np.concatenate((Two_Features_Test, Cosine_Sim_Array_Test), axis=1)
print(Cos_Three_Features_Test.shape)

#### Using logistic regression

In [ ]:
from sklearn import linear_model
clf = linear_model.LogisticRegression()
y_pred_cos_three_lr = clf.fit(Cos_Three_Features, Target_Array).predict(Cos_Three_Features_Test)
y_pred_list_cos_three_lr = y_pred_cos_three_lr.tolist()
#%store y_pred_list_cos_three_lr > y_pred_list_cos_three_lr.txt

In [ ]:
total = Cos_Three_Features_Test.shape[0]
mislabelled = (Target_Array_Test != y_pred_cos_three_lr).sum()
print("Number of mislabeled points out of a total %d points : %d" % (Cos_Three_Features_Test.shape[0],(Target_Array_Test != y_pred_cos_three_lr).sum()))

In [ ]:
acc = (total-mislabelled) / total
print("Accuracy {}".format(acc))

In [ ]:
### Using Many features along with cosine similarity

from scipy.sparse import hstack
Cos_Many_Features = hstack([Many_Features,Cosine_Sim_Array])
print(Many_Features.shape, Cosine_Sim_Array.shape)

print(Cos_Many_Features.shape)

In [ ]:
Cos_Many_Features_Test = np.concatenate((Many_Features_Test, Cosine_Sim_Array_Test), axis=1)
print(Cos_Many_Features_Test.shape)

In [ ]:
from sklearn import linear_model
clf = linear_model.LogisticRegression()
y_pred_cos_many_lr = clf.fit(Cos_Many_Features, Target_Array).predict(Cos_Many_Features_Test)
y_pred_list_cos_many_lr = y_pred_cos_many_lr.tolist()

In [ ]:
total = Cos_Many_Features_Test.shape[0]
mislabelled = (Target_Array_Test != y_pred_cos_many_lr).sum()
print("Number of mislabeled points out of a total %d points : %d" % (Cos_Many_Features_Test.shape[0],(Target_Array_Test != y_pred_cos_many_lr).sum()))

In [ ]:
acc = (total-mislabelled) / total
print("Accuracy {}".format(acc))

#### Using ridge regression

In [ ]:
from sklearn import linear_model
# Regularized regression
clf = linear_model.RidgeClassifier(1.0, fit_intercept=False) # MSE + 1.0 l2
y_pred_cos_three_rr = clf.fit(Cos_Three_Features, Target_Array).predict(Cos_Three_Features_Test)
y_pred_list_cos_three_rr = y_pred_cos_three_rr.tolist()
#%store y_pred_list_cos_three_lr > y_pred_list_cos_three_lr.txt

total = Cos_Three_Features_Test.shape[0]
mislabelled = (Target_Array_Test != y_pred_cos_three_rr).sum()
print("Number of mislabeled points out of a total %d points : %d" % (Cos_Three_Features_Test.shape[0],(Target_Array_Test != y_pred_cos_three_rr).sum()))

acc = (total-mislabelled) / total
print("Accuracy {}".format(acc))

### Using Many features along with cosine similarity

from scipy.sparse import hstack
Cos_Many_Features = hstack([Many_Features,Cosine_Sim_Array])
print(Many_Features.shape, Cosine_Sim_Array.shape)

print(Cos_Many_Features.shape)

Cos_Many_Features_Test = np.concatenate((Many_Features_Test, Cosine_Sim_Array_Test), axis=1)
print(Cos_Many_Features_Test.shape)

from sklearn import linear_model
clf = linear_model.RidgeClassifier(1.0, fit_intercept=False) # MSE + 1.0 l2
y_pred_cos_many_rr = clf.fit(Cos_Many_Features, Target_Array).predict(Cos_Many_Features_Test)
y_pred_list_cos_many_rr = y_pred_cos_many_rr.tolist()

total = Cos_Many_Features_Test.shape[0]
mislabelled = (Target_Array_Test != y_pred_cos_many_rr).sum()
print("Number of mislabeled points out of a total %d points : %d" % (Cos_Many_Features_Test.shape[0],(Target_Array_Test != y_pred_cos_many_rr).sum()))

acc = (total-mislabelled) / total
print("Accuracy {}".format(acc))

#### Using SVM

In [ ]:
from sklearn import linear_model
clf = linear_model.SGDClassifier()
y_pred_cos_three_svm = clf.fit(Cos_Three_Features, Target_Array).predict(Cos_Three_Features_Test)
y_pred_list_cos_three_svm = y_pred_cos_three_svm.tolist()
#%store y_pred_list_cos_three_lr > y_pred_list_cos_three_lr.txt

total = Cos_Three_Features_Test.shape[0]
mislabelled = (Target_Array_Test != y_pred_cos_three_svm).sum()
print("Number of mislabeled points out of a total %d points : %d" % (Cos_Three_Features_Test.shape[0],(Target_Array_Test != y_pred_cos_three_svm).sum()))

acc = (total-mislabelled) / total
print("Accuracy {}".format(acc))

### Using Many features along with cosine similarity

from scipy.sparse import hstack
Cos_Many_Features = hstack([Many_Features,Cosine_Sim_Array])
print(Many_Features.shape, Cosine_Sim_Array.shape)

print(Cos_Many_Features.shape)

Cos_Many_Features_Test = np.concatenate((Many_Features_Test, Cosine_Sim_Array_Test), axis=1)
print(Cos_Many_Features_Test.shape)

from sklearn import linear_model
clf = linear_model.SGDClassifier()
y_pred_cos_many_svm = clf.fit(Cos_Many_Features, Target_Array).predict(Cos_Many_Features_Test)
y_pred_list_cos_many_svm = y_pred_cos_many_svm.tolist()

total = Cos_Many_Features_Test.shape[0]
mislabelled = (Target_Array_Test != y_pred_cos_many_svm).sum()
print("Number of mislabeled points out of a total %d points : %d" % (Cos_Many_Features_Test.shape[0],(Target_Array_Test != y_pred_cos_many_svm).sum()))

acc = (total-mislabelled) / total
print("Accuracy {}".format(acc))

### Euclidean distance + TF-IDF

In [ ]:
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.translate.bleu_score import sentence_bleu
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import euclidean_distances


stop = set(stopwords.words('english'))
ps = PorterStemmer()
vectorizer = TfidfVectorizer()
euclid_sim_vector = []

#all_words = defaultdict(int)
#prem_hyp_pair = []
for j in range(len(pp.data)):
    sentence1 = pp.data[j]['sentence1']
    sentence1 = unicodedata.normalize('NFKD', sentence1)
    premise = [ps.stem(i) for i in sentence1.lower().split() if i not in stop]
    #for word in premise:
    #    all_words[word] += 1
    sentence2 = pp.data[j]['sentence2']
    sentence2 = unicodedata.normalize('NFKD', sentence2)
    hypothesis = [ps.stem(i) for i in sentence2.lower().split() if i not in stop]
    #for word in hypothesis:
    #    all_words[word] += 1
    allsentences = []
    allsentences.append(' '.join(premise))
    allsentences.append(' '.join(hypothesis))
    #prem_hyp_pair.append((premise,hypothesis))
    
    sparse_matrix = vectorizer.fit_transform(allsentences)
    doc_term_matrix = sparse_matrix.todense()
    
    euclid_sim_vector.append(euclidean_distances(doc_term_matrix, doc_term_matrix)[0][1])

In [ ]:
Euclid_Sim_Array = np.array(euclid_sim_vector)
Euclid_Sim_Array = Euclid_Sim_Array[:, np.newaxis]

Euc_Three_Features = np.concatenate((Two_Features, Euclid_Sim_Array), axis=1)
print(Euc_Three_Features.shape)
print(Euclid_Sim_Array.shape, Two_Features.shape)

In [ ]:
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.translate.bleu_score import sentence_bleu
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import euclidean_distances


stop = set(stopwords.words('english'))
ps = PorterStemmer()
vectorizer = TfidfVectorizer()
euclid_sim_vector_test = []

#all_words = defaultdict(int)
#prem_hyp_pair = []
X =[]
for j in range(len(tt.data)):
    sentence1 = tt.data[j]['sentence1']
    sentence1 = unicodedata.normalize('NFKD', sentence1)
    premise = [ps.stem(i) for i in sentence1.lower().split() if i not in stop]
    #for word in premise:
    #    all_words[word] += 1
    sentence2 = tt.data[j]['sentence2']
    sentence2 = unicodedata.normalize('NFKD', sentence2)
    hypothesis = [ps.stem(i) for i in sentence2.lower().split() if i not in stop]
    #for word in hypothesis:
    #    all_words[word] += 1
    allsentences = []
    allsentences.append(' '.join(premise))
    allsentences.append(' '.join(hypothesis))
    #prem_hyp_pair.append((premise,hypothesis))
    
    sparse_matrix = vectorizer.fit_transform(allsentences)
    doc_term_matrix = sparse_matrix.todense()
    
    euclid_sim_vector_test.append(euclidean_distances(doc_term_matrix, doc_term_matrix)[0][1])

In [ ]:
Euclid_Sim_Array_Test = np.array(euclid_sim_vector_test)
Euclid_Sim_Array_Test = Euclid_Sim_Array_Test[:, np.newaxis]

Euc_Three_Features_Test = np.concatenate((Two_Features_Test, Euclid_Sim_Array_Test), axis=1)
print(Euc_Three_Features_Test.shape)

#### Using logistic regression

In [ ]:
from sklearn import linear_model
clf = linear_model.LogisticRegression()
y_pred_euc_three_lr = clf.fit(Euc_Three_Features, Target_Array).predict(Euc_Three_Features_Test)
#y_pred_list_cos_three_lr = y_pred_cos_three_lr.tolist()
#%store y_pred_list_cos_three_lr > y_pred_list_cos_three_lr.txt

In [ ]:
total = Euc_Three_Features_Test.shape[0]
mislabelled = (Target_Array_Test != y_pred_euc_three_lr).sum()
print("Number of mislabeled points out of a total %d points : %d" % (Euc_Three_Features_Test.shape[0],(Target_Array_Test != y_pred_euc_three_lr).sum()))

In [ ]:
acc = (total-mislabelled) / total
print("Accuracy {}".format(acc))

In [ ]:
### Using Many features along with cosine similarity

from scipy.sparse import hstack
Euc_Many_Features = hstack([Many_Features,Euclid_Sim_Array])
print(Many_Features.shape, Euclid_Sim_Array.shape)

print(Cos_Many_Features.shape)

In [ ]:
Euc_Many_Features_Test = np.concatenate((Many_Features_Test, Euclid_Sim_Array_Test), axis=1)
print(Euc_Many_Features_Test.shape)

In [ ]:
from sklearn import linear_model
clf = linear_model.LogisticRegression()
y_pred_euc_many_lr = clf.fit(Euc_Many_Features, Target_Array).predict(Euc_Many_Features_Test)
y_pred_list_euc_many_lr = y_pred_euc_many_lr.tolist()

In [ ]:
total = Euc_Many_Features_Test.shape[0]
mislabelled = (Target_Array_Test != y_pred_euc_many_lr).sum()
print("Number of mislabeled points out of a total %d points : %d" % (Euc_Many_Features_Test.shape[0],(Target_Array_Test != y_pred_euc_many_lr).sum()))

In [ ]:
acc = (total-mislabelled) / total
print("Accuracy {}".format(acc))

### Jaccard on pair

In [ ]:
def Jaccard(A,B):
    return len(A.intersection(B)) / len(A.union(B))

In [ ]:
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.translate.bleu_score import sentence_bleu
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import euclidean_distances


stop = set(stopwords.words('english'))
ps = PorterStemmer()
jaccard_scores = []

#all_words = defaultdict(int)
#prem_hyp_pair = []
for j in range(len(pp.data)):
    sentence1 = pp.data[j]['sentence1']
    sentence1 = unicodedata.normalize('NFKD', sentence1)
    premise = [ps.stem(i) for i in sentence1.lower().split() if i not in stop]
    #for word in premise:
    #    all_words[word] += 1
    sentence2 = pp.data[j]['sentence2']
    sentence2 = unicodedata.normalize('NFKD', sentence2)
    hypothesis = [ps.stem(i) for i in sentence2.lower().split() if i not in stop]
    jaccard_scores.append(Jaccard(set(premise), set(hypothesis)))


In [ ]:
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.translate.bleu_score import sentence_bleu
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import euclidean_distances

stop = set(stopwords.words('english'))
ps = PorterStemmer()

jaccard_scores_test =[]

for j in range(len(tt.data)):
    sentence1 = tt.data[j]['sentence1']
    sentence1 = unicodedata.normalize('NFKD', sentence1)
    premise = [ps.stem(i) for i in sentence1.lower().split() if i not in stop]
    sentence2 = tt.data[j]['sentence2']
    sentence2 = unicodedata.normalize('NFKD', sentence2)
    hypothesis = [ps.stem(i) for i in sentence2.lower().split() if i not in stop]
    jaccard_scores_test.append(Jaccard(set(premise), set(hypothesis)))
    

In [ ]:
Sim_Array = np.array(jaccard_scores)
Sim_Array = Sim_Array[:, np.newaxis]

Jacc_Three_Features = np.concatenate((Two_Features, Sim_Array), axis=1)
Jacc_Many_Features = hstack([Many_Features, Sim_Array])

Sim_Array_Test = np.array(jaccard_scores_test)
Sim_Array_Test = Sim_Array_Test[:, np.newaxis]
print(Sim_Array_Test.shape)

Jacc_Three_Features_Test = np.concatenate((Two_Features_Test, Sim_Array_Test), axis=1)
Jacc_Many_Features_Test = np.concatenate((Many_Features_Test, Sim_Array_Test), axis = 1)


In [ ]:
from sklearn import linear_model
clf = linear_model.LogisticRegression()
y_pred_jacc_three_lr = clf.fit(Jacc_Three_Features, Target_Array).predict(Jacc_Three_Features_Test)


In [ ]:
total = Jacc_Three_Features_Test.shape[0]
mislabelled = (Target_Array_Test != y_pred_jacc_three_lr).sum()
print("Number of mislabeled points out of a total %d points : %d" % (Jacc_Three_Features_Test.shape[0],(Target_Array_Test != y_pred_jacc_three_lr).sum()))

In [ ]:
from sklearn import linear_model
clf = linear_model.LogisticRegression()
y_pred_jacc_many_lr = clf.fit(Jacc_Many_Features, Target_Array).predict(Jacc_Many_Features_Test)


total = Jacc_Many_Features_Test.shape[0]
mislabelled = (Target_Array_Test != y_pred_jacc_many_lr).sum()
print("Number of mislabeled points out of a total %d points : %d" % (Jacc_Many_Features_Test.shape[0],(Target_Array_Test != y_pred_jacc_many_lr).sum()))

In [ ]:
acc = (total-mislabelled) / total
print("Accuracy {}".format(acc))

In [ ]:

Jacc_Cos_Many_Features = hstack([Jacc_Many_Features, Cosine_Sim_Array])
Jacc_Cos_Many_Features_Test = np.concatenate((Jacc_Many_Features_Test, Cosine_Sim_Array_Test), axis = 1)


In [ ]:
from sklearn import linear_model
clf = linear_model.LogisticRegression(C=0.1)
y_pred_jacc_cos_many_lr = clf.fit(Jacc_Cos_Many_Features, Target_Array).predict(Jacc_Cos_Many_Features_Test)


total = Jacc_Cos_Many_Features_Test.shape[0]
mislabelled = (Target_Array_Test != y_pred_jacc_cos_many_lr).sum()
print("Number of mislabeled points out of a total %d points : %d" % (Jacc_Cos_Many_Features_Test.shape[0],(Target_Array_Test != y_pred_jacc_cos_many_lr).sum()))

In [ ]:
acc = (total-mislabelled) / total
print("Accuracy {}".format(acc))